In [ ]:
# Import required packages
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7, DEFAULT_SOLVER_PARAM_M3

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7)
solver.path_following(
    load_time="1111215818",
    num_traj=100,
    shrink_ratio=0,
    enable_evaluation=True,
    enable_plot=False,
)

In [ ]:
import numpy as np
import pandas as pd
from time import time
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver
from paik.solver import max_joint_angle_change

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

P = solver.sample_P_path(load_time="1111215818")

num_traj = 100
num_steps = P.shape[0]

l2_err_arr = np.empty((num_traj, num_steps))
ang_err_arr = np.empty((num_traj, num_steps))
mjac_arr = np.empty((num_traj,))
runtime_arr = np.empty((num_traj))

begin = time()
for i in range(num_traj):
    sol, l2_err_arr[i], ang_err_arr[i], _, _, runtime = ik_solver.solve_n_poses(
        P, latent_scale=0.01, refine_solutions=False, return_detailed=True
    )  # type: ignore
    mjac_arr[i] = max_joint_angle_change(sol)  # type: ignore

df = pd.DataFrame(
    {
        "l2_err": l2_err_arr.mean(axis=1),
        "ang_err": ang_err_arr.mean(axis=1),
        "mjac": mjac_arr,
    }
)
print(df.describe())
print(f"avg_inference_time: {round((time() - begin) / num_traj, 3)}")